In [1]:
import pandas as pd
import rdflib
from rdflib.namespace import RDF
from rdflib import Namespace
from rdflib import URIRef, Literal

In [2]:
global g2
g2=rdflib.Graph()
g2.parse('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_Union_2.nt',format='nt')
print(len(g2))

11108


In [3]:
#************Change Path to files************
independent_dept_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/independent_dept _new_working.csv", header=None) 

In [4]:
independent_dept_data.columns=['Cat1','Cat2','Org1','Cat3','Org2']

In [5]:
#Fucntion:Changing plural to singular
def change_cat(col_nam,df):
    singular_plural_data=pd.read_csv("C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Singular_Plural.csv") 
    for i in range(singular_plural_data.shape[0]):
        val=singular_plural_data.iloc[i,0]
        new_val=singular_plural_data.iloc[i,1]
        df.loc[(df[col_nam]).apply(lambda x: x.strip())==val,col_nam]=new_val
    return(df)

In [6]:
#Fucntion Call:Changing plural to singular
independent_dept_data=change_cat('Cat1',independent_dept_data)
independent_dept_data=change_cat('Cat2',independent_dept_data)
independent_dept_data=change_cat('Cat3',independent_dept_data)

In [7]:
global org
global bhorg
global skos
global bhcatres
global bhorgres

org=Namespace("http://www.w3.org/ns/org#")
bhorg=Namespace("http://semanticwebindia.in/bharathi/ont/org/")
skos=Namespace("http://www.w3.org/2004/02/skos/core#")
bhcatres=Namespace("http://semanticwebindia.in/bharathi/res/cat/")
bhorgres=Namespace("http://semanticwebindia.in/bharathi/res/org/")

In [8]:
pd.set_option('display.max_colwidth', -1)#housekeeping

In [9]:
#finding n_org max
global n_org
org_list=[]
for s,p,o in g2.triples((None, RDF.type, bhorg.Organisation)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    num=int(str_s_uri[pos+1:len(str_s_uri)])
    num_s=str(num)
    num_s=num_s[1:]
    num_i=int(num_s)
    org_list.append(num_i) #list of n_org identifier values
org_list.sort() #sorting the list
n_org=org_list[-1] #the current maximum value of n_org

#No need to find n_cat max, just one skos:category- topGOT
global n_cat
n_cat=0
cat_list=[]
for s,p,o in g2.triples((None, skos.prefLabel, None)):
    str_s_uri=str(s)
    pos=str_s_uri.rfind('/')
    if str_s_uri[pos+1:len(str_s_uri)]!='topOT':
        num=int(str_s_uri[pos+1:len(str_s_uri)])
        num_s=str(num)
        num_s=num_s[1:]
        num_i=int(num_s)
        cat_list.append(num_i) #list of n_org identifier values
cat_list.sort() #sorting the list
n_cat=cat_list[-1] #the current maximum value of n_org

In [10]:
print(n_cat)
print(n_org)

6
1415


In [11]:
def org_scrub(org_n):
    l_scrub=[]
#     ls_scrub[0]=""
#     ls_scrub[1]=""
#     ls_scrub[2]=""
    val=org_n
    val=str(val)
    val=val.replace('[',"")
    val=val.replace(']',"")
    if len(val)==0:
        return(l_scrub)
    #print(val)
    pos_url=val.find('http')
    #print(pos_url)
    if pos_url!=-1:
        val_1=val[:pos_url]+'|'+val[pos_url:]
        pos_end_url=val_1[pos_url+1:].find("'")
        val_2=val_1[:pos_url+1+pos_end_url]+'|'+val_1[pos_url+1+pos_end_url:]
    else:
        sch_ch=val[0]
        pos_end=val[1:].find(sch_ch)
        #print(val[:pos_end+2])
        val_2=val[:pos_end+2]+"|"+val[pos_end+2:]
        #print(val_2)
    org_ls=val_2.split('|')
    
    for i in range(len(org_ls)):
        ele=org_ls[i]
        ele_ls=list(ele)
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[0] in ["'",'"',',',' '] and len(ele_ls)>0:
                ele_ls.pop(0)
                #print(ele_ls)
            if len(ele_ls)>0:
                if ele_ls[0] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        clean_c=1
        while clean_c>0 and len(ele_ls)>0:
            if ele_ls[-1] in ["'",'"',',',' ']:
                ele_ls.pop(-1)
            if len(ele_ls)>0:
                if ele_ls[-1] in ["'",'"',',',' ']:
                    clean_c=1
                else:
                    clean_c=0
            else:
                clean_c=0
        ele_j=''.join(ele_ls)
        l_scrub.append(ele_j)
    
#     if ls_scrub[0]=="":
#         ls_scrub[0]=ele_j
#     elif ls_scrub[1]=="" and ele_j.find('http')>-1:
#         ls_scrub[1]=ele_j
#     elif ls_scrub[2]=="":
#         ls_scrub[2]=ele_j
    
    return(l_scrub)

In [12]:
#create fetch function for cat_id
def fetch_n_cat(cat):
    global n_cat
    global g2
    cat_id=""
    for s,p,o in g2.triples((None, skos.prefLabel,Literal(cat))):
        cat_id=s
    if cat_id=="":
        bhcatres="http://semanticwebindia.in/bharathi/res/cat/"
        n_cat+=1
        cat_id=URIRef(bhcatres+str('2')+str(n_cat))
        #adding relevant triples
#         g2.add((cat_id,skos.broader, URIRef(bhcatres+'topOT')))
#         g2.add((URIRef(bhcatres+'topOT'),skos.broaderOf,cat_id))
        g2.add((cat_id,RDF.type,skos.Concept))
        g2.add((cat_id,skos.prefLabel, Literal(cat)))
        g2.add((cat_id,skos.altLabel, Literal(cat)))
        g2.add((cat_id,skos.inScheme,URIRef(bhcatres+'OrganisationType')))
    return(cat_id)

#create fetch function for org_id
def fetch_n_org(org_nam,org_url="",org_abb=""):
    global g2
    global n_org
    org_id=""
    for s,p,o in g2.triples((None, bhorg.hasName,Literal(org_nam))):
        org_id=s
    if org_id=="":
        bhorgres="http://semanticwebindia.in/bharathi/res/org/"
        n_org+=1
        org_id=URIRef(bhorgres+str('1')+str(n_org))
        g2.add((org_id, bhorg.hasName, Literal(org_nam)))
        g2.add((org_id, bhorg.hasAlternateName, Literal(org_nam)))
        
        if org_url!="":
            g2.add((org_id, bhorg.hasURL, Literal(org_url)))
        if org_abb!="":
            g2.add((org_id, bhorg.hasAbbreviation, Literal(org_abb)))
        g2.add((org_id,RDF.type,bhorg.Organisation))
        g2.add((org_id,org.identifier,Literal(n_org)))
        g2.add((org_id,org.hasOrganizationType,URIRef(bhcatres+str('21')))) #skos(Government) :bhcatres/21
    return(org_id)

In [13]:
#create fetch_cat_broader for broader relation
def fetch_cat_broader(cat1_id,cat2_id):
    global g2
    if not (cat2_id, skos.broader,cat1_id) in g2:
        g2.add((cat2_id, skos.broader,cat1_id))
    if not (cat1_id, skos.broaderOf,cat2_id) in g2:
        g2.add((cat1_id, skos.broaderOf,cat2_id))
        
#create fetch_org_subOrg for sub-organization relation
def fetch_org_subOrg(org1_id,org2_id):
    global g2
    if not (org1_id,org.hasSubOrganization,org2_id) in g2:
        g2.add((org1_id,org.hasSubOrganization,org2_id))
    if not (org2_id,org.subOrganizationOf,org1_id) in g2:
        g2.add((org2_id,org.subOrganizationOf,org1_id))

In [14]:
#Iteration over union_data rows
for i in range(independent_dept_data.shape[0]):
    
    cat1=independent_dept_data.iloc[i,0].strip()
    cat2=independent_dept_data.iloc[i,1].strip()
    org1=independent_dept_data.iloc[i,2]
    cat3=independent_dept_data.iloc[i,3]
    org2=independent_dept_data.iloc[i,4]
        
    #check if cat1_id already exists, else create
    cat1_id=fetch_n_cat(cat1)
    
    #check if cat2_id already exists, else create
    cat2_id=fetch_n_cat(cat2)
    
    #check if cat3_id already exists, else create
    cat3_id=fetch_n_cat(cat3)
    
    #check if ((cat1 broader cat21)), ((cat21 broaderOf cat1)) exist, else create
    cat_govt=URIRef(bhcatres+'21')
    fetch_cat_broader(cat_govt,cat1_id)
    #check if ((cat1 broader cat2)), ((cat2 broaderOf cat2)) exist, else create
    fetch_cat_broader(cat1_id,cat2_id)#*****************CHECK/CHANGE THIS*****************#
    #check if ((cat2 broader cat3)), ((cat2 broaderOf cat3)) exist, else create
    fetch_cat_broader(cat2_id,cat3_id)#*****************CHECK/CHANGE THIS*****************#
    
    #check if org1_id already exists, else create
    ##clean and return list
    org1_ls=org_scrub(org1)
    org_nam=""
    org_url=""
    org_abb=""
    if len(org1_ls)==1:
        org_nam=org1_ls[0]
    elif len(org1_ls)==2:
        org_nam=org1_ls[0]
        if org1_ls[1].find('http')>0:
            org_url=org1_ls[1]
        else:
            org_abb=org1_ls[1]
    else:
        org_nam=org1_ls[0]
        org_url=org1_ls[1]
        org_abb=org1_ls[2]
        
    org1_id=fetch_n_org(org_nam,org_url,org_abb)
    
    #create tuple org1, org:classification, cat2_id
    g2.add((org1_id,org.classification,cat2_id)) 
    
    #check if org2_id already exists, else create
    org2_ls=org_scrub(org2)
    if (len(org2_ls))>0: 
        if len(org2_ls)==1:
            org_nam=org2_ls[0]
#         elif len(org2_ls)==2:
#             org_nam=org2_ls[0]
#             org_url=org2_ls[1]
        elif len(org2_ls)==2:
            org_nam=org2_ls[0]
            if org2_ls[1].find('http')>0:
                org_url=org2_ls[1]
            else:
                org_abb=org2_ls[1]
        else:
            org_nam=org2_ls[0]
            org_url=org2_ls[1]
            org_abb=org2_ls[2]
        org2_id=fetch_n_org(org_nam,org_url,org_abb)
        #create tuple org2, org:classificatoin, cat2_id
        g2.add((org2_id,org.classification,cat3_id))
        fetch_org_subOrg(org1_id,org2_id)

In [15]:
len(g2)

11807

In [16]:
g2.serialize(format='nt', destination='C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/Graph_IndependentDept_3.nt')

In [17]:
#Testing
for s,p,o in g2.triples((None, bhorg.hasAlternateName,Literal("Indian Space Research Organisation (ISRO)"))):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/org/11462 http://semanticwebindia.in/bharathi/ont/org/hasAlternateName Indian Space Research Organisation (ISRO)


In [18]:
#Testing ISRO
val=independent_dept_data.iloc[93,4]
print(val)
val_ls=org_scrub(val)
print(val_ls)

['Indian Space Research Organisation (ISRO)' 'http://www.isro.gov.in' 'ISRO ']
['Indian Space Research Organisation (ISRO)', 'http://www.isro.gov.in', 'ISRO']


In [19]:
u=URIRef("http://semanticwebindia.in/bharathi/res/org/11477")
for s,p,o in g2.triples((u,None,None)):
    print(s,p,o)

http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/ns/org#classification http://semanticwebindia.in/bharathi/res/cat/213
http://semanticwebindia.in/bharathi/res/org/11477 http://semanticwebindia.in/bharathi/ont/org/hasAlternateName ISRO Satellite Centre(ISAC) Bengaluru
http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/ns/org#identifier 1477
http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/ns/org#hasOrganizationType http://semanticwebindia.in/bharathi/res/cat/21
http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/ns/org#subOrganizationOf http://semanticwebindia.in/bharathi/res/org/11461
http://semanticwebindia.in/bharathi/res/org/11477 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://semanticwebindia.in/bharathi/ont/org/Organisation
http://semanticwebindia.in/bharathi/res/org/11477 http://semanticwebindia.in/bharathi/ont/org/hasName ISRO Satellite Centre(ISAC) Bengaluru
http://semanticwebindia.in/bharathi/res/or

In [20]:
independent_dept_data.to_csv('C:/Users/SESA456791/Desktop/Ontology-Bh/Files/Outputs/independent_dept_data_w_changes.csv',encoding='utf8',index=False)

In [21]:
for s,p,o in g2.triples((None,bhorg.hasName,None)):
    print(o + "|") 

Kendriya Vidyalaya Jalgaon, Maharashtra|
Department of Humanities and Social Sciences, IIT Bombay|
Kenndriya Viyalaya Saharsa, Bihar|
Tourism|
Central University of Bihar (CUB)|
Jawahar Navodaya Vidyalaya(JNV), Durgapur, Burdwan, West Bengal|
Kendriya Vidyalaya Sangathan (KVS), Hyderabad Region|
Kendriya Vidyalaya IIT, Chennai Region, Tamil Nadu|
Institute of Co-Operative Management (ICM), Imphal, Manipur|
C-DAC - Advanced Computing Training School (ACTS), Pune|
Institute of Hotel Managment, Bangalore|
National Commission for Minority Educational Institutions (NCMEI)|
Indian Pharmacopoeia Commission (IPC)|
Kendriya Vidyalaya No.5, Gwalior, Madhya Pradesh|
National Fire Service College|
National Centre for Compositional Characterisation of Materials Department of Atomic Energy (DAE)|
Institute of Forest Genetics and Tree Breeding (IFGTB)|
University of Delhi|
Jawahar Navodaya Vidyalaya (JNVS), Jajpur, Odisha|
Centre for Development of Advanced Computing (C-DAC), Mumbai|
Kendriya Vidyala